In [1]:
from PIL import Image, ImageDraw
import numpy as np

In [2]:
## Helper functions; helpful global parameters.

size = (1920*2,1800*2)
base_path = "/Users/dhruv13/Desktop/new/"

def get_center():
    """
        Returns the center of the point cloud to be drawn.
        The center is randomly generated by a Gaussian with 
        mean as the center of the image and sigma (100., 100.).
    """
    center = np.round(np.random.normal((size[0]/2., size[1]/2.), (100., 100.), 2))
    return center

def get_sigma():
    """
        Returns the spread of the point cloud. Controls how much the dots are spread around the center.
        Keeping this fixed as I think spread messes with numerosity.
    """
    return (500,500)

def gen_single_image(center, sigma, num, radius):
    """
        Generate and return a single image.

        Parameters:
            name - Name of the file.
            center - Center of the point cloud to be drawn.
            sigma - Spread of the point cloud to be drawn (think gaussian).
            num1 - Number of points to be drawn on the left image.
            num2 - Number of points to be drawn on the right image.
            radius - radius of each point.
    """
    im = Image.new('RGB', size)
    draw = ImageDraw.Draw(im)

    white = (255,255,255)
    black = (0, 0, 0)

    i = 0
    while i < num:
        circle_pos = np.round(np.random.normal(center, sigma, 2))
        circle_box = (circle_pos[0], circle_pos[1], circle_pos[0] + 2*radius, circle_pos[1] + 2*radius)
        draw.ellipse(circle_box, fill = white, outline = black)
        i += 1
    return im

def gen_test_image(name, center, sigma, num1, num2, radius):
    """
        Generates a stimulus image for the experiment.

        Parameters:
            name - Name of the file.
            center - Center of the point cloud to be drawn.
            sigma - Spread of the point cloud to be drawn (think gaussian).
            num1 - Number of points to be drawn on the left image.
            num2 - Number of points to be drawn on the right image.
            radius - radius of each point.
    """
    im_left = gen_single_image(center, sigma, num1, radius)
    im_right = gen_single_image(center, sigma, num2, radius)
    
    gray = (127, 127, 127)
    
    new_size = (size[0] * 2 + 150, size[1] + 100)
    im = Image.new('RGB', new_size)
    
    im.paste(gray)
    im.paste(im_left, (50, 50))
    im.paste(im_right, (100 + size[0], 50))
    
    im.save(base_path + "%s.png"%(name), "PNG")

def store_single_image(name, center, sigma, num, radius):
    """
        Store a single image at the base path.

        Parameters:
            name - Name of the file.
            center - Center of the point cloud to be drawn.
            sigma - spread of the point cloud to be drawn (think gaussian).
            num - number of points to be drawn.
            radius - radius of each point.
    """
    im = gen_single_image(center, sigma, num, radius)
    im.save(base_path + "%s.png"%(name), "PNG")
    
def save_conditions_file(name, image, n, correct):
    """
        Create the conditions file for the experiment. It is saved at the base path.
        
        Parameters:
            name - name of the file to be saved
            image - list containing names of the stimulus image files.
            n - list containing the left and right number of points in the stimulus image.
            correct - list of correct responses for the stimuli.
    """
    f = open(base_path + name + ".csv", "w")
    f.write("file,n1,n2,correct\n")
    
    if len(image) != len(n) or len(image) != len(correct):
        raise ValueError("Length of lists do not match.")
    
    for i in range(len(image)):
        f.write("%s,%s,%s,%s\n"%(str(image[i]), str(n[i][0]), str(n[i][1]), str(correct[i])))
    
    f.close()

In [3]:
s = """
        Goal: Study Weber's law for Numerosity - The sense of quantity

        Weber's Law: The threshold for detection of change from a perceptual quantity is directly proportional to 
                     the magnitude of the quantity.

        Numerosity: The sense of visual quantity going beyond counting.

        Experiment design

        - Image Stimulus: Two images presented side by side, black background, white dots (A Point Cloud, if you will).

        - Task : Identify the image with more number of dots, and accordingly respond within 15 seconds with:
                a) left arrow key - if you think that the left image has more dots.
                b) right arrow key - if you think that the right image has more dots.
                c) down arrow key - if you think that both have the same number.

        - First thoughts : I need to calculate the weber fraction. For this, I need to have a "left" image,
                           the actual number of dots from which the change needs to be measured. To make a weber 
                           fraction conclusion, I guess I will need at least 6 data points: 1 low, 3 mid, 2 high. 
                           Each will have to be coupled with a certain number of images with increasing number of 
                           dots to precisely capture the threshold for change detection from it.
                           This essentially helps me fix the number of trials and adjust time for presentation of the 
                           stimulus. I also have to think about how to randomize the spread, and probably control for 
                           it.

        - Analysis : Go from "yes" - "no" responses to quantification of the threshold for stimulus change. 
                     Then calculate the Weber fraction.

        // Parameters for the experiment.
        - Sample number of points (6): [8, 16, 24, 40, 56, 80, 112]; deltas: [8, 8, 16, 16, 24, 32]
        - Spread: X (Keep X = 500 throughout)
        - Numbers of Comparisons for each sample: 4, 6, 8, 10, 15, 15, 20
        - Total comparisons made per subject: 78

        Issues:

        1) Order of presentation: 'Random' or 'Not Random'? 
            - Random presentation can counter the counting issue. Sequential presentation will let the subject 
              count both sides and make a more informed guess. Thus, a random sequence should be used.
            - The sequence should be preserved across subjects, which prevents the noise introduced by the 
              adaptation to the task.

        2) Control for spread and counting: How does spread of the dots affect the estimate?
            - Introduce a small set of same number of images to the set with different spread. Could be done towards 
              the end of the experiment.

        3) Number of subjects: Can't help this, has to be ~6.
    """

## Actual image generation begins here.

radius = 15 # Radius of each point.

center = get_center()
sigma = get_sigma()

samples = [8, 16, 24, 40, 56, 80, 112]
comparisons = [4, 6, 8, 10, 15, 15, 20]

i = 0

image_paths = []
n = []
correct = []

for left, iterations in zip(samples, comparisons):
    for right in range(iterations):
        i += 1
        right = left + right

        # Flip a coin and randomize which goes where 
        if np.round(np.random.uniform(0, 2)) == 1.0:
            right, left = left, right
        gen_test_image("im%d"%(i), center, sigma, left, right, radius)
        
        image_paths.append(base_path + "im%d.png"%(i))
        
        n.append((left, right))
        
        if left > right:
            correct.append("left")
        elif right > left:
            correct.append("right")
        else:
            correct.append("down")

# Randomize conditions at this point, don't want to present sequentially!
indices = np.arange(len(image_paths)) # array of indices
np.random.shuffle(indices) # a random shuffle

# now, arrange each condition according to the shuffle.
image_paths = [image_paths[idx] for idx in indices]
n = [n[idx] for idx in indices]
correct = [correct[idx] for idx in indices]

save_conditions_file("conditions", image_paths, n, correct)